In [1]:
import os
from os import listdir
from os.path import isfile, join
import glob
import string
import operator
import csv
import pandas as pd
import calendar

In [2]:
homedir = os.path.expanduser("~")
path_to_data = homedir+"\\Assignment 3\\Data"
print(path_to_data)

C:\Users\priya\Assignment 3\Data


### Getting data from CSV file 

In [3]:
output_dist={}
for subdir,dirs, files in os.walk(path_to_data):
    #print(files)
    for file in files:
        if file == 'vehicle_collisions.csv':
            #print(file)
            csv_data1=pd.read_csv(os.path.join(subdir,file))
print(csv_data1.head(2))

,UNIQUE KEY,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,...,VEHICLE 1 TYPE,VEHICLE 2 TYPE,VEHICLE 3 TYPE,VEHICLE 4 TYPE,VEHICLE 5 TYPE,VEHICLE 1 FACTOR,VEHICLE 2 FACTOR,VEHICLE 3 FACTOR,VEHICLE 4 FACTOR,VEHICLE 5 FACTOR
0,3146911,1/1/15,0:20,QUEENS,11358.0,40.751847,-73.787862,"(40.7518471, -73.787862)",47 AVENUE,193 STREET,...,SPORT UTILITY/STATION WAGON,NaN,NaN,NaN,NaN,TRAFFIC CONTROL DISREGARDED,NaN,NaN,NaN,NaN
1,3146180,1/1/15,0:20,NaN,NaN,40.771289,-73.946693,"(40.7712888, -73.9466928)",NaN,NaN,...,PASSENGER VEHICLE,NaN,NaN,NaN,NaN,ANIMALS ACTION,NaN,NaN,NaN,NaN


In [4]:
new_df = csv_data1[["BOROUGH","DATE","VEHICLE 1 TYPE","VEHICLE 2 TYPE","VEHICLE 3 TYPE","VEHICLE 4 TYPE","VEHICLE 5 TYPE"]].copy()

,BOROUGH,DATE,VEHICLE 1 TYPE,VEHICLE 2 TYPE,VEHICLE 3 TYPE,VEHICLE 4 TYPE,VEHICLE 5 TYPE
0,QUEENS,1/1/15,SPORT UTILITY/STATION WAGON,NaN,NaN,NaN,NaN
1,NaN,1/1/15,PASSENGER VEHICLE,NaN,NaN,NaN,NaN


In [5]:
new_df["DATE"]= pd.to_datetime(new_df.DATE)
new_df["YEAR"] = new_df.DATE.dt.year
print(new_df.head(5))

,BOROUGH,DATE,VEHICLE 1 TYPE,VEHICLE 2 TYPE,VEHICLE 3 TYPE,VEHICLE 4 TYPE,VEHICLE 5 TYPE,YEAR
0,QUEENS,2015-01-01,SPORT UTILITY/STATION WAGON,NaN,NaN,NaN,NaN,2015
1,NaN,2015-01-01,PASSENGER VEHICLE,NaN,NaN,NaN,NaN,2015
2,BROOKLYN,2015-01-01,PASSENGER VEHICLE,UNKNOWN,NaN,NaN,NaN,2015
3,BROOKLYN,2015-01-01,BUS,PASSENGER VEHICLE,NaN,NaN,NaN,2015
4,NaN,2015-01-01,UNKNOWN,PASSENGER VEHICLE,NaN,NaN,NaN,2015


# Inserting 1 in place of vehical name and zero for NaN

In [6]:
new_df['VEHICLE 1 TYPE'] = new_df['VEHICLE 1 TYPE'].str.replace('.*','1')
new_df['VEHICLE 1 TYPE'] = new_df['VEHICLE 1 TYPE'].fillna(0)

new_df['VEHICLE 2 TYPE'] = new_df['VEHICLE 2 TYPE'].str.replace('.*','1')
new_df['VEHICLE 2 TYPE'] = new_df['VEHICLE 2 TYPE'].fillna(0)

new_df['VEHICLE 3 TYPE'] = new_df['VEHICLE 3 TYPE'].str.replace('.*','1')
new_df['VEHICLE 3 TYPE'] = new_df['VEHICLE 3 TYPE'].fillna(0)

new_df['VEHICLE 4 TYPE'] = new_df['VEHICLE 4 TYPE'].str.replace('.*','1')
new_df['VEHICLE 4 TYPE'] = new_df['VEHICLE 4 TYPE'].fillna(0)

new_df['VEHICLE 5 TYPE'] = new_df['VEHICLE 5 TYPE'].str.replace('.*','1')
new_df['VEHICLE 5 TYPE'] = new_df['VEHICLE 5 TYPE'].fillna(0)

print(new_df.head(5))

,BOROUGH,DATE,VEHICLE 1 TYPE,VEHICLE 2 TYPE,VEHICLE 3 TYPE,VEHICLE 4 TYPE,VEHICLE 5 TYPE,YEAR
0,QUEENS,2015-01-01,1,0,0,0,0,2015
1,NaN,2015-01-01,1,0,0,0,0,2015
2,BROOKLYN,2015-01-01,1,1,0,0,0,2015
3,BROOKLYN,2015-01-01,1,1,0,0,0,2015
4,NaN,2015-01-01,1,1,0,0,0,2015


### Getting only data after 2015

In [7]:
new_df = new_df[(new_df.YEAR >= 2015)].dropna(subset=["BOROUGH"])
print(new_df.head(5))

,BOROUGH,DATE,VEHICLE 1 TYPE,VEHICLE 2 TYPE,VEHICLE 3 TYPE,VEHICLE 4 TYPE,VEHICLE 5 TYPE,YEAR
0,QUEENS,2015-01-01,1,0,0,0,0,2015
2,BROOKLYN,2015-01-01,1,1,0,0,0,2015
3,BROOKLYN,2015-01-01,1,1,0,0,0,2015
5,BROOKLYN,2015-01-01,1,1,1,1,0,2015
6,QUEENS,2015-01-01,1,1,1,0,0,2015


## Counting total for each column 

In [8]:
vehical_type1_df = new_df[["BOROUGH"]].copy()
vehical_type1_df["one_vehical_involved"] = new_df["VEHICLE 1 TYPE"].astype(str).astype(int)
vehical_type1_df = pd.DataFrame(vehical_type1_df.groupby("BOROUGH").one_vehical_involved.sum().reset_index())

,BOROUGH,one_vehical_involved
0,BRONX,45599
1,BROOKLYN,104174
2,MANHATTAN,82801
3,QUEENS,89140
4,STATEN ISLAND,13943


In [9]:
vehical_type2_df = new_df[["BOROUGH"]].copy()
vehical_type2_df["two_vehical_involved"] = new_df["VEHICLE 2 TYPE"].astype(str).astype(int)
vehical_type2_df = pd.DataFrame(vehical_type2_df.groupby("BOROUGH").two_vehical_involved.sum().reset_index())

,BOROUGH,two_vehical_involved
0,BRONX,36972
1,BROOKLYN,86799
2,MANHATTAN,69483
3,QUEENS,76178
4,STATEN ISLAND,11600


In [10]:
vehical_type3_df = new_df[["BOROUGH"]].copy()
vehical_type3_df["three_vehical_involved"] = new_df["VEHICLE 3 TYPE"].astype(str).astype(int)
vehical_type3_df = pd.DataFrame(vehical_type3_df.groupby("BOROUGH").three_vehical_involved.sum().reset_index())

,BOROUGH,three_vehical_involved
0,BRONX,2587
1,BROOKLYN,6592
2,MANHATTAN,2525
3,QUEENS,5918
4,STATEN ISLAND,924


In [11]:
vehical_type4_df = new_df[["BOROUGH"]].copy()
vehical_type4_df["four_vehical_involved"] = new_df["VEHICLE 4 TYPE"].astype(str).astype(int)
vehical_type4_df = pd.DataFrame(vehical_type4_df.groupby("BOROUGH").four_vehical_involved.sum().reset_index())

,BOROUGH,four_vehical_involved
0,BRONX,625
1,BROOKLYN,1612
2,MANHATTAN,501
3,QUEENS,1420
4,STATEN ISLAND,165


In [12]:
vehical_type5_df = new_df[["BOROUGH"]].copy()
vehical_type5_df["five_vehical_involved"] = new_df["VEHICLE 5 TYPE"].astype(str).astype(int)
vehical_type5_df = pd.DataFrame(vehical_type5_df.groupby("BOROUGH").five_vehical_involved.sum().reset_index())

,BOROUGH,five_vehical_involved
0,BRONX,173
1,BROOKLYN,461
2,MANHATTAN,129
3,QUEENS,314
4,STATEN ISLAND,32


## Merging all dataframes to get single data frame

In [13]:
final_df = pd.merge(vehical_type1_df,vehical_type2_df,on ='BOROUGH' , how='inner')
final_df.columns = ["BOROUGH","one_vehical_involved","two_vehical_involved"]

final_df = pd.merge(final_df,vehical_type3_df,on ='BOROUGH' , how='inner')
final_df.columns = ["BOROUGH","one_vehical_involved","two_vehical_involved","three_vehical_involved"]

final_df = pd.merge(final_df,vehical_type4_df,on ='BOROUGH' , how='inner')
final_df.columns = ["BOROUGH","one_vehical_involved","two_vehical_involved","three_vehical_involved","four_vehical_involved"]

final_df = pd.merge(final_df,vehical_type5_df,on ='BOROUGH' , how='inner')
final_df.columns = ["BOROUGH","one_vehical_involved","two_vehical_involved","three_vehical_involved","four_vehical_involved","five_vehical_involved"]

#save_to_CSV()
print(final_df.head(2))

,BOROUGH,one_vehical_involved,two_vehical_involved,three_vehical_involved,four_vehical_involved,five_vehical_involved
0,BRONX,45599,36972,2587,625,173
1,BROOKLYN,104174,86799,6592,1612,461


# Subtracting the colums to get exact count

In [14]:
final_df["four_vehical_involved"] = final_df["four_vehical_involved"] - final_df["five_vehical_involved"]


In [15]:
final_df["three_vehical_involved"] = (final_df["three_vehical_involved"] - final_df["four_vehical_involved"])-final_df["five_vehical_involved"]


In [16]:
final_df["two_vehical_involved"] = ((final_df["two_vehical_involved"]-final_df["three_vehical_involved"])-final_df["four_vehical_involved"])-final_df["five_vehical_involved"]                     


In [17]:
final_df["one_vehical_involved"]=(((final_df["one_vehical_involved"]-final_df["two_vehical_involved"])-final_df["three_vehical_involved"])-final_df["four_vehical_involved"])- final_df["five_vehical_involved"]


In [ ]:
output_df = final_df
output_df["More_Vehicals_involved"] = output_df["four_vehical_involved"] + output_df["five_vehical_involved"]
output_df.drop('four_vehical_involved', axis=1, inplace=True)
output_df.drop('five_vehical_involved', axis=1, inplace=True)

print(output_df.head(5))

In [19]:
def save_to_CSV():
    if not os.path.exists(homedir+"\\Assignment 3\\Output\\Question1_part2"):
        os.makedirs(homedir+"\\Assignment 3\\Output\\Question1_part2")
    with open(homedir+'\\Assignment 3\\Output\\Question1_part2\\Q1P2.csv', 'w') as myfile:
        myfile.write(output_df.to_csv(index=False))

In [ ]:
save_to_CSV()